## Segmenting and Clustering Neighbourhoods in Toronto

### Week 3 pt 1


In [1]:
!conda install -c anaconda beautifulsoup4 --yes
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c anaconda lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 anaconda
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current

In [2]:
import requests 
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np
import pandas as pd

In [3]:
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [27]:
# using bs4

def tabelabs4(cname, cols):
    pagina = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(pagina, 'lxml')
    tabela = soup.find("table", class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in tabela.find_all("th")]
    data = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
            for tr in tabela.find_all("tr")]
    data = [row for row in data if len(row) == cols]
    crudf = pd.DataFrame(data, columns=header)
    return crudf

In [29]:
# verifying
crutrt = tabelabs4("wikitable",3)

print(crutrt.shape)

(288, 3)


In [31]:
# clean up - ignore not assigned
trtpost = crutrt[~crutrt['Borough'].isin(['Not assigned'])]
trtpost = trtpost.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

In [37]:
# replacing missing neighborhoods for boroughs
trtpost.loc[trtpost['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = trtpost['Borough']

In [40]:
# combining neighborhoods
trtpost = trtpost.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
trtpost

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


In [42]:
trtpost.shape

(103, 3)